In [3]:
import sys
import io
import shutil
import sys
import datetime
import subprocess
import sys
import math
import base64
from time import gmtime, strftime
import random, string
import time
import base64
import uuid
import socket
import os
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import MilvusClient
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from dotenv import load_dotenv
load_dotenv(verbose=True)
import json
import requests
from pymilvus import model
from pymilvus.model.dense import SentenceTransformerEmbeddingFunction
from datetime import datetime, timedelta

model = SentenceTransformerEmbeddingFunction('all-MiniLM-L6-v2',device='cpu' )

DIMENSION = 384 
TOKEN = os.environ.get("ZILLIZ_TOKEN")
API_KEY = os.environ.get("API_KEY")
MILVUS_URL = os.environ.get("SERVERLESS_VECTORDB_URI")
slack_token = os.environ.get("SLACK_BOT_TOKEN")
client = WebClient(token=slack_token)

milvus_client = MilvusClient( uri=MILVUS_URL, token=TOKEN )

OAQ3_COLLECTION_NAME = "openaqmeasurements"
TODAYS_DATE = str( datetime.today().strftime('%Y-%m-%d') ) 
YESTERDAYS_DATE = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')

details = ""
fulllocation = ""
data = []

## schema

schema = milvus_client.create_schema(
    enable_dynamic_field=False
)

schema.add_field(field_name='id', datatype=DataType.INT64, is_primary=True, auto_id=True)
schema.add_field(field_name='locationId', datatype=DataType.INT32)
schema.add_field(field_name='location', datatype=DataType.VARCHAR, max_length=255)
schema.add_field(field_name='parameter', datatype=DataType.VARCHAR, max_length=255)
schema.add_field(field_name="value", datatype=DataType.FLOAT)
schema.add_field(field_name='datelocal', datatype=DataType.VARCHAR, max_length=255)
schema.add_field(field_name="unit", datatype=DataType.VARCHAR, max_length=255)
schema.add_field(field_name="latitude", datatype=DataType.FLOAT)
schema.add_field(field_name="longitude", datatype=DataType.FLOAT)
schema.add_field(field_name="country", datatype=DataType.VARCHAR, max_length=255)
schema.add_field(field_name="city", datatype=DataType.VARCHAR, max_length=255)
schema.add_field(field_name="isMobile", datatype=DataType.VARCHAR, max_length=12)
schema.add_field(field_name="isAnalysis", datatype=DataType.VARCHAR, max_length=12)
schema.add_field(field_name='entity', datatype=DataType.VARCHAR, max_length=255)
schema.add_field(field_name='sensorType', datatype=DataType.VARCHAR, max_length=255)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=DIMENSION)
schema.add_field(field_name="details",  datatype=DataType.VARCHAR, max_length=8000)
schema.add_field(field_name="fulllocation",  datatype=DataType.VARCHAR, max_length=2000)

index_params = milvus_client.prepare_index_params()

index_params.add_index(
    field_name="id",
    index_type="STL_SORT"
)

index_params.add_index(
    field_name="vector",
    index_type="IVF_FLAT",
    metric_type="L2",
    params={"nlist": 100}
)

if not milvus_client.has_collection(collection_name = OAQ3_COLLECTION_NAME):
    milvus_client.create_collection(
        collection_name = OAQ3_COLLECTION_NAME,
        schema=schema,
        index_params=index_params
    )
    
    res = milvus_client.get_load_state(
        collection_name = OAQ3_COLLECTION_NAME
    )
    print(res)


/Users/timothyspann/Downloads/code/milvusvenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [38]:
# -----------------------------------------------------------------------------
# Search Milvus for that vector and filter by a label
# -----------------------------------------------------------------------------
results = milvus_client.search(
    OAQ3_COLLECTION_NAME,
    data=[model(details)],
    filter='location like "%H%"',
    output_fields=["location", "details", "parameter","value", "datelocal","fulllocation"],
    limit=5
)

# -----------------------------------------------------------------------------
# Iterate through last five results and display metadata and image
# -----------------------------------------------------------------------------
for result in results:
    for hit in result[:5]:
        location = hit["entity"]["location"]
        details = hit["entity"]["details"]
        parameter = hit["entity"]["parameter"]
        value = hit["entity"]["value"]
        datelocal = hit["entity"]["datelocal"]
        fulllocation = hit["entity"]["fulllocation"]

        print(f"{location} {details} {parameter}={value} {datelocal} {fulllocation}" )

HAMPTON Current Air Quality Reading for no2 is 0.0064 ppm for Location 1120: HAMPTON, None, US @ 37.10372999999999,-76.38702 at 2024-09-10T23:00:00-04:00. Is Mobile: False Is Analysis: None Entity: Governmental Organization Sensor Type: reference grade no2=0.006399999838322401 2024-09-10T23:00:00-04:00 Location 1120: HAMPTON, None, US @ 37.10372999999999,-76.38702
HAMPTON Current Air Quality Reading for no2 is 0.0064 ppm for Location 1120: HAMPTON, None, US @ 37.10372999999999,-76.38702 at 2024-09-10T23:00:00-04:00. Is Mobile: False Is Analysis: None Entity: Governmental Organization Sensor Type: reference grade no2=0.006399999838322401 2024-09-10T23:00:00-04:00 Location 1120: HAMPTON, None, US @ 37.10372999999999,-76.38702
HAMPTON Current Air Quality Reading for no2 is 0.0064 ppm for Location 1120: HAMPTON, None, US @ 37.10372999999999,-76.38702 at 2024-09-10T23:00:00-04:00. Is Mobile: False Is Analysis: None Entity: Governmental Organization Sensor Type: reference grade no2=0.0063999

In [39]:
res = milvus_client.search(
    collection_name=OAQ3_COLLECTION_NAME,
    data=[model(details)],
    limit=25, 
    group_by_field="location", 
    output_fields=["location", "parameter","value"],
)

for result in res:
    for hit in result[:25]:
        location = hit["entity"]["location"]
        parameter = hit["entity"]["parameter"]
        value = hit["entity"]["value"]

        print(f"{location} [{parameter}={value}]" )
        print( '-------------------------')



HAMPTON [no2=0.006399999838322401]
-------------------------
Jerome Mack [no2=0.006800000090152025]
-------------------------
Joe Neal [no2=0.007000000216066837]
-------------------------
Roosevelt [no2=0.006399999838322401]
-------------------------
Reseda [no2=0.0066999997943639755]
-------------------------
Chester [no2=0.0020000000949949026]
-------------------------
Los Angeles - N. Mai [no2=0.006399999838322401]
-------------------------
LYNN [no2=0.002899999963119626]
-------------------------
TRINITY [no2=0.006200000178068876]
-------------------------
Rutland [no2=0.007899999618530273]
-------------------------
Jenkins Avenue [no2=0.012000000104308128]
-------------------------
ROLAND [no2=0.0035000001080334187]
-------------------------
TracyAP [no2=0.002099999925121665]
-------------------------
EDMUNDSTON QUEEN ST. [no2=0.0017999999690800905]
-------------------------
Reno4 [no2=0.002899999963119626]
-------------------------
WeymouthFR [no2=0.010999999940395355]
----------

In [41]:
res = milvus_client.query(
    collection_name=OAQ3_COLLECTION_NAME,
    filter='location like "T%" && parameter == "pm25"',
    group_by_field="location", 
    output_fields=["location", "parameter","value"],
    limit=10
)

for result in res:
    print(result)


{'location': 'Twin Falls PM25', 'parameter': 'pm25', 'value': 20.0, 'id': 452466995772398524}
{'location': 'Twin Falls PM25', 'parameter': 'pm25', 'value': 22.0, 'id': 452466995772398525}
{'location': 'Twin Falls PM25', 'parameter': 'pm25', 'value': 19.0, 'id': 452466995772398526}
{'location': 'Twin Falls PM25', 'parameter': 'pm25', 'value': 19.0, 'id': 452466995772398527}
{'location': 'Twin Falls PM25', 'parameter': 'pm25', 'value': 20.0, 'id': 452466995772398528}
{'location': 'Twin Falls PM25', 'parameter': 'pm25', 'value': 16.0, 'id': 452466995772398529}
{'location': 'Twin Falls PM25', 'parameter': 'pm25', 'value': 13.0, 'id': 452466995772398530}
{'location': 'Twin Falls PM25', 'parameter': 'pm25', 'value': 20.0, 'id': 452466995772398531}
{'location': 'Twin Falls PM25', 'parameter': 'pm25', 'value': 32.0, 'id': 452466995772398532}
{'location': 'Twin Falls PM25', 'parameter': 'pm25', 'value': 38.0, 'id': 452466995772398533}


In [42]:

res = milvus_client.query(
    collection_name=OAQ3_COLLECTION_NAME,
    output_fields=["count(*)"]
)

print(res)


data: ["{'count(*)': 302000}"] , extra_info: {'cost': 6}


In [37]:
res = milvus_client.search(
    collection_name=OAQ3_COLLECTION_NAME,
    data=[model(details)],
    limit=25, 
    output_fields=["location", "parameter","value", "datelocal", "details"],
)

for result in res:
    for hit in result[:25]:
        print(hit)

{'id': 452498223019153336, 'distance': 3.0157204946590355e-06, 'entity': {'value': 0.006399999838322401, 'datelocal': '2024-09-10T23:00:00-04:00', 'location': 'HAMPTON', 'parameter': 'no2'}}
{'id': 452466995783615434, 'distance': 3.0157204946590355e-06, 'entity': {'value': 0.006399999838322401, 'datelocal': '2024-09-10T23:00:00-04:00', 'location': 'HAMPTON', 'parameter': 'no2'}}
{'id': 452466995772411865, 'distance': 0.0007470485288649797, 'entity': {'value': 0.006399999838322401, 'datelocal': '2024-09-10T08:00:00-04:00', 'location': 'HAMPTON', 'parameter': 'no2'}}
{'id': 452466995772440021, 'distance': 0.0007470485288649797, 'entity': {'value': 0.006399999838322401, 'datelocal': '2024-09-10T08:00:00-04:00', 'location': 'HAMPTON', 'parameter': 'no2'}}
{'id': 452466995772440030, 'distance': 0.0036377732176333666, 'entity': {'value': 0.006599999964237213, 'datelocal': '2024-09-09T23:00:00-04:00', 'location': 'HAMPTON', 'parameter': 'no2'}}
{'id': 452466995772411874, 'distance': 0.0036377

In [43]:
# python search code
res = milvus_client.search(
  collection_name="openaqmeasurements", # Collection name
    data=[model(details)],
  search_params={
    "metric_type": "L2",
    "params": {"nprobe":1}, # Search parameters
  }, # Search parameters
  limit=250, # Max. number of search results to return
  output_fields=["id","locationId","location","parameter","value","datelocal","unit","latitude","longitude","country","city","isMobile","isAnalysis","entity","sensorType","details","fulllocation"], # Fields to return in the search results
  consistency_level="Eventually"
)

print(res)


data: ["[{'id': 452466995772440021, 'distance': 2.7749608761951094e-06, 'entity': {'fulllocation': 'Location 1120: HAMPTON, None, US @ 37.10372999999999,-76.38702', 'id': 452466995772440021, 'latitude': 37.103729248046875, 'longitude': -76.38701629638672, 'isMobile': 'False', 'isAnalysis': 'None', 'locationId': 1120, 'location': 'HAMPTON', 'country': 'US', 'city': 'None', 'datelocal': '2024-09-10T08:00:00-04:00', 'details': 'Current Air Quality Reading for no2 is 0.0064 ppm for Location 1120: HAMPTON, None, US @ 37.10372999999999,-76.38702 at 2024-09-10T08:00:00-04:00. Is Mobile: False Is Analysis: None Entity: Governmental Organization Sensor Type: reference grade', 'parameter': 'no2', 'value': 0.006399999838322401, 'unit': 'ppm', 'entity': 'Governmental Organization', 'sensorType': 'reference grade'}}, {'id': 452466995772411865, 'distance': 2.7749608761951094e-06, 'entity': {'fulllocation': 'Location 1120: HAMPTON, None, US @ 37.10372999999999,-76.38702', 'id': 452466995772411865, 'l